In [ ]:
import pandas as pd
import csv

In [ ]:
data_tweets = pd.read_csv('/content/stock_tweets.csv')
data_tweets.head()

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."


In [ ]:
df_amzn = data_tweets[data_tweets['Stock Name'] == 'AAPL']
df_amzn.head()

,Date,Tweet,Stock Name,Company Name
55958,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...,AAPL,Apple Inc.
55959,2022-09-29 20:37:01+00:00,After trading for 9+ years \n\nThis is how to ...,AAPL,Apple Inc.
55960,2022-09-29 20:19:43+00:00,Not something you see very often. \n-2 ATR mo...,AAPL,Apple Inc.
55961,2022-09-29 20:13:48+00:00,$AAPL was down almost 5% today. Rare to see it...,AAPL,Apple Inc.
55962,2022-09-29 19:50:00+00:00,$AAPL APPLE JUST FIRED ITS HEAD OF PROCUREMENT...,AAPL,Apple Inc.


In [ ]:
df_amzn['Date'] = pd.to_datetime(df_amzn['Date']).dt.date
#list(df_amzn.groupby('Date')['Tweet'])
pdata = df_amzn.groupby('Date')['Tweet']
print(pdata.head())

55958    $NIO just because I'm down money doesn't mean ...
55959    After trading for 9+ years \n\nThis is how to ...
55960    Not something you see very often.  \n-2 ATR mo...
55961    $AAPL was down almost 5% today. Rare to see it...
55962    $AAPL APPLE JUST FIRED ITS HEAD OF PROCUREMENT...
                               ...                        
61007                         I need 1,000 shares of $AAPL
61008    Returns over the last 10 Years:\n\nBitcoin $BT...
61009    All about this trendline now on $AAPL, continu...
61010    This thread is just a broad overview of the ba...
61011    In 2020, Tim Cook, CEO of $AAPL, earned $265,0...
Name: Tweet, Length: 1779, dtype: object


<ipython-input-26-f4e3ccdda78b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amzn['Date'] = pd.to_datetime(df_amzn['Date']).dt.date


In [ ]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
import re
from textblob import TextBlob

In [ ]:
def get_tweet_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [ ]:
all_tweets = []
for date, twt in pdata:
    #print (date)
    tweets_day = {}
    tweets_data = []
    tweets_day['Date'] = date
    count = 0
    for t in twt:
        parsedTweet = {}
        #parsedTweet['text'] = t
        parsedTweet['sentiment'] = get_tweet_sentiment(t)
        tweets_data.append(parsedTweet)
        count = count + 1
    #tweets_day['data'] = tweets_data
    #print (tweets_day)

    ptweets = [tweet for tweet in tweets_data if tweet['sentiment'] == 'positive']
    tweets_day['Positive'] = 100*len(ptweets)/len(tweets_data)
    ntweets = [tweet for tweet in tweets_data if tweet['sentiment'] == 'negative']
    tweets_day['Negative'] = 100*len(ntweets)/len(tweets_data)
    tweets_day['Neutral'] = 100*(len(tweets_data) - len(ntweets) - len(ptweets))/len(tweets_data)
    tweets_day['Count'] = count
    print (tweets_day)
    all_tweets.append(tweets_day)

{'Date': datetime.date(2021, 9, 30), 'Positive': 28.571428571428573, 'Negative': 28.571428571428573, 'Neutral': 42.857142857142854, 'Count': 7}
{'Date': datetime.date(2021, 10, 1), 'Positive': 54.54545454545455, 'Negative': 18.181818181818183, 'Neutral': 27.272727272727273, 'Count': 11}
{'Date': datetime.date(2021, 10, 2), 'Positive': 25.0, 'Negative': 25.0, 'Neutral': 50.0, 'Count': 4}
{'Date': datetime.date(2021, 10, 3), 'Positive': 100.0, 'Negative': 0.0, 'Neutral': 0.0, 'Count': 1}
{'Date': datetime.date(2021, 10, 4), 'Positive': 50.0, 'Negative': 0.0, 'Neutral': 50.0, 'Count': 10}
{'Date': datetime.date(2021, 10, 5), 'Positive': 40.0, 'Negative': 20.0, 'Neutral': 40.0, 'Count': 5}
{'Date': datetime.date(2021, 10, 6), 'Positive': 60.0, 'Negative': 0.0, 'Neutral': 40.0, 'Count': 10}
{'Date': datetime.date(2021, 10, 7), 'Positive': 62.5, 'Negative': 0.0, 'Neutral': 37.5, 'Count': 8}
{'Date': datetime.date(2021, 10, 8), 'Positive': 85.71428571428571, 'Negative': 0.0, 'Neutral': 14.285

In [ ]:
print(all_tweets)

[]


In [ ]:
data_amzn_df = pd.DataFrame(all_tweets)
data_amzn_df.head()

,Date,Positive,Negative,Neutral,Count
0,2021-09-30,28.571429,28.571429,42.857143,7
1,2021-10-01,54.545455,18.181818,27.272727,11
2,2021-10-02,25.000000,25.000000,50.000000,4
3,2021-10-03,100.000000,0.000000,0.000000,1
4,2021-10-04,50.000000,0.000000,50.000000,10


In [ ]:
data_amzn_df.to_csv('aapl_tweets_sentiments.csv', index=False)

In [ ]:
data_stock = pd.read_csv('stock_yfinance_data.csv')
data_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,2021-09-30,260.333344,263.043335,258.333344,258.493347,258.493347,53868000,TSLA
1,2021-10-01,259.466675,260.260010,254.529999,258.406677,258.406677,51094200,TSLA
2,2021-10-04,265.500000,268.989990,258.706665,260.510010,260.510010,91449900,TSLA
3,2021-10-05,261.600006,265.769989,258.066681,260.196655,260.196655,55297800,TSLA
4,2021-10-06,258.733337,262.220001,257.739990,260.916656,260.916656,43898400,TSLA


In [ ]:
data_amzn_stock = data_stock[data_stock['Stock Name'] == 'AAPL']
data_amzn_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
1764,2021-09-30,143.660004,144.380005,141.279999,141.500000,140.478485,89056700,AAPL
1765,2021-10-01,141.899994,142.919998,139.110001,142.649994,141.620163,94639600,AAPL
1766,2021-10-04,141.759995,142.210007,138.270004,139.139999,138.135513,98322000,AAPL
1767,2021-10-05,139.490005,142.240005,139.360001,141.110001,140.091278,80861100,AAPL
1768,2021-10-06,139.470001,142.149994,138.369995,142.000000,140.974869,83221100,AAPL


In [ ]:
data_amzn_stock['Date'] = pd.to_datetime(data_amzn_stock['Date'])
data_amzn_df['Date'] = pd.to_datetime(data_amzn_df['Date'])

f4 = data_amzn_df[data_amzn_df['Date'].isin(data_amzn_stock['Date'])]
f4.head()

<ipython-input-35-086cc9175482>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_amzn_stock['Date'] = pd.to_datetime(data_amzn_stock['Date'])


,Date,Positive,Negative,Neutral,Count
0,2021-09-30,28.571429,28.571429,42.857143,7
1,2021-10-01,54.545455,18.181818,27.272727,11
4,2021-10-04,50.000000,0.000000,50.000000,10
5,2021-10-05,40.000000,20.000000,40.000000,5
6,2021-10-06,60.000000,0.000000,40.000000,10


In [ ]:
df_merged = pd.merge(data_amzn_stock,f4, on='Date')
df_merged.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name,Positive,Negative,Neutral,Count
0,2021-09-30,143.660004,144.380005,141.279999,141.500000,140.478485,89056700,AAPL,28.571429,28.571429,42.857143,7
1,2021-10-01,141.899994,142.919998,139.110001,142.649994,141.620163,94639600,AAPL,54.545455,18.181818,27.272727,11
2,2021-10-04,141.759995,142.210007,138.270004,139.139999,138.135513,98322000,AAPL,50.000000,0.000000,50.000000,10
3,2021-10-05,139.490005,142.240005,139.360001,141.110001,140.091278,80861100,AAPL,40.000000,20.000000,40.000000,5
4,2021-10-06,139.470001,142.149994,138.369995,142.000000,140.974869,83221100,AAPL,60.000000,0.000000,40.000000,10


In [ ]:
len(df_merged)

252

In [ ]:
df_merged_amzn = df_merged
df_merged_amzn.to_csv('sentiments_aapl_year.csv')

In [ ]:
data_AMZN = pd.read_csv('/content/sentiments_amzn_year.csv')